In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
SC=StandardScaler()
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBRFClassifier,XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# for dealing with imbalanced data
from imblearn.under_sampling import NearMiss,RandomUnderSampler
from imblearn.over_sampling import SMOTE,RandomOverSampler,SMOTENC
from imblearn.combine import SMOTETomek

In [4]:

## Lets test Chisquare test of Independence to find out which all features are of importnce.
# Assume Significance level =0.05
# Ho: There is no significant dependency between Categorical feature(X1 Or X2...or Xn) and Label:target
# H1: There is no  dependency between Categorical feature(X1 Or X2...or Xn) and Label:target
# If the Actual p-value computed from Chisquare test is < 0.05 then we reject the NULL hypothesis
        #(Means there is dependency betwen X and y and we can retain the feature)
# If the Actual p-value computed from Chisquare test is >= 0.05 then we fail to reject the NULL hypothesis
         #(Means there is no dependency betwen X and y and we can drop the feature)
# The basic requirement of Chisquare test is: All variables should be Categorical.
from scipy.stats import chi2_contingency

train_chi=pd.read_csv(r'C:\Saroj_Official\AI\Kaggle\Categorical-Featuer-Encoding-Challange-2\train.csv').copy()
train_chi=train_chi.astype('object').copy()
train_chi.dropna(inplace=True)
# Select all catagorical columns except target
categorical_columns=list(train_chi.select_dtypes(include='object').drop(columns=['id','target']).columns)



chi2_check = []
for i in categorical_columns:
    ch2 , p_value , df, exp_freq=chi2_contingency(pd.crosstab(train_chi[i],train_chi['target']))
    if p_value < 0.05:
        chi2_check.append('Reject Null Hypothesis: Retain the Feature:')
    else:
        chi2_check.append('Fail to Reject Null Hypothesis: Drop the Feature:')
        
result = pd.DataFrame(chi2_check,columns=['Remarks'])
result.insert(0,'Feature',categorical_columns)
del train_chi
result

,Feature,Remarks
0,bin_0,Reject Null Hypothesis: Retain the Feature:
1,bin_1,Reject Null Hypothesis: Retain the Feature:
2,bin_2,Reject Null Hypothesis: Retain the Feature:
3,bin_3,Fail to Reject Null Hypothesis: Drop the Feature:
4,bin_4,Reject Null Hypothesis: Retain the Feature:
5,nom_0,Reject Null Hypothesis: Retain the Feature:
6,nom_1,Reject Null Hypothesis: Retain the Feature:
7,nom_2,Reject Null Hypothesis: Retain the Feature:
8,nom_3,Reject Null Hypothesis: Retain the Feature:
9,nom_4,Reject Null Hypothesis: Retain the Feature:


In [6]:
train=pd.read_csv(r'C:\Saroj_Official\AI\Kaggle\Categorical-Featuer-Encoding-Challange-2\train.csv')
test=pd.read_csv(r'C:\Saroj_Official\AI\Kaggle\Categorical-Featuer-Encoding-Challange-2\test.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 25 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      600000 non-null  int64  
 1   bin_0   582106 non-null  float64
 2   bin_1   581997 non-null  float64
 3   bin_2   582070 non-null  float64
 4   bin_3   581986 non-null  object 
 5   bin_4   581953 non-null  object 
 6   nom_0   581748 non-null  object 
 7   nom_1   581844 non-null  object 
 8   nom_2   581965 non-null  object 
 9   nom_3   581879 non-null  object 
 10  nom_4   581965 non-null  object 
 11  nom_5   582222 non-null  object 
 12  nom_6   581869 non-null  object 
 13  nom_7   581997 non-null  object 
 14  nom_8   582245 non-null  object 
 15  nom_9   581927 non-null  object 
 16  ord_0   581712 non-null  float64
 17  ord_1   581959 non-null  object 
 18  ord_2   581925 non-null  object 
 19  ord_3   582084 non-null  object 
 20  ord_4   582070 non-null  object 
 21  ord_5   58

In [7]:
%%time
# Lets fill in the Null value for train
for i in train.columns:
    if train[i].isnull().sum()>0:
        train[i].fillna(train[i].mode()[0],inplace=True)
        
# Lets fill in the Null value for train
for i in test.columns:
    if test[i].isnull().sum()>0:
        test[i].fillna(test[i].mode()[0],inplace=True)
# Check for Null Values
print('total number of null values in train data are: ',train.isnull().sum().sum())
print('total number of null values in test data are: ',test.isnull().sum().sum())

total number of null values in train data are:  0
total number of null values in test data are:  0
Wall time: 2.39 s


In [8]:
# Chek if the train data is balanced or imbalanced
train.target.value_counts()

0    487677
1    112323
Name: target, dtype: int64

In [9]:
%%time

# Preprocessing of Train and test Data together
#Concatenate Train and Test Data with a new column to identify Train (Type=0) VS Test Data(Type=1)  
train['Type']=pd.DataFrame(np.zeros(len(train)).astype(int)) ## 0 for train and 1 for test data
test['Type']=pd.DataFrame(np.ones(len(test)).astype(int))

print('Original Train Data shape:{} and Test Data shape:{}'.format(train.shape,test.shape))
traintest=pd.concat([train.drop(columns=['target','id']),test.drop(columns=['id'])])
print('traintest shape after combining Train and Test ',traintest.shape)
label=train.target

Original Train Data shape:(600000, 26) and Test Data shape:(400000, 25)
traintest shape after combining Train and Test  (1000000, 24)
Wall time: 417 ms


In [10]:
train.target.value_counts()

0    487677
1    112323
Name: target, dtype: int64

In [11]:
traintest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 399999
Data columns (total 24 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   bin_0   1000000 non-null  float64
 1   bin_1   1000000 non-null  float64
 2   bin_2   1000000 non-null  float64
 3   bin_3   1000000 non-null  object 
 4   bin_4   1000000 non-null  object 
 5   nom_0   1000000 non-null  object 
 6   nom_1   1000000 non-null  object 
 7   nom_2   1000000 non-null  object 
 8   nom_3   1000000 non-null  object 
 9   nom_4   1000000 non-null  object 
 10  nom_5   1000000 non-null  object 
 11  nom_6   1000000 non-null  object 
 12  nom_7   1000000 non-null  object 
 13  nom_8   1000000 non-null  object 
 14  nom_9   1000000 non-null  object 
 15  ord_0   1000000 non-null  float64
 16  ord_1   1000000 non-null  object 
 17  ord_2   1000000 non-null  object 
 18  ord_3   1000000 non-null  object 
 19  ord_4   1000000 non-null  object 
 20  ord_5   1000000 non-null 

In [12]:
%%time
# Lets apply oneHotEncoding
traintest_OHE = pd.get_dummies(traintest, columns=traintest.columns, drop_first=True, sparse=True)
train_ohe = traintest_OHE[traintest_OHE.Type_1==0].drop(columns='Type_1')
test_ohe = traintest_OHE[traintest_OHE.Type_1==1].drop(columns='Type_1')

#Convert dataframe to spare matrix
train_ohe = train_ohe.sparse.to_coo().tocsr()
test_ohe = test_ohe.sparse.to_coo().tocsr()

Wall time: 5min 49s


In [9]:
train_ohe

<600000x5678 sparse matrix of type '<class 'numpy.longlong'>'
	with 10230886 stored elements in Compressed Sparse Row format>

In [13]:
#Lets try to balance the dataset by combination of over and under sampling- SMOTETomek
imb=RandomUnderSampler(random_state=42)
#imb=SMOTETomek(random_state=42)
X=train_ohe
y=label
train_ohe,label=imb.fit_sample(X,y)

In [14]:
print(train_ohe.shape)
print(label.shape)

(224646, 5678)
(224646,)


In [15]:
features=train_ohe

In [16]:
#Lets perform Cross validations considering all features and see what could be the best score

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score,classification_report,confusion_matrix
#from sklearn import metrics

def stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y):
    global df_model_selection
    
    skf = StratifiedKFold(n_splits, random_state=12,shuffle=True)
    
    weighted_f1_score = []
    #print(skf.split(X,y))
    for train_index, test_index in skf.split(X,y):
        X_train, X_test = X[train_index], X[test_index] 
        y_train, y_test = y[train_index], y[test_index]
        
        
        model_obj.fit(X_train, y_train)
        test_ds_predicted = model_obj.predict( X_test )      
        weighted_f1_score.append(round(f1_score(y_true=y_test, y_pred=test_ds_predicted , average='weighted'),2))
        
    sd_weighted_f1_score = np.std(weighted_f1_score, ddof=1)
    range_of_f1_scores = "{}-{}".format(min(weighted_f1_score),max(weighted_f1_score))    
    df_model_selection = pd.concat([df_model_selection,pd.DataFrame([[process,model_name,sorted(weighted_f1_score),range_of_f1_scores,sd_weighted_f1_score]], columns =COLUMN_NAMES) ])

In [17]:
%%time
COLUMN_NAMES = ["Process","Model Name", "F1 Scores","Range of F1 Scores","Std Deviation of F1 Scores"]
df_model_selection = pd.DataFrame(columns=COLUMN_NAMES)

features=train_ohe

process='ALl Features'
n_splits = 5
#X=SC.fit_transform(features)
X=features
y=label

# # 1.Naive Bayes
# model_NB=BernoulliNB()
# model_obj=model_NB
# model_name='Naive Bayes'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 2.Logistic Regression
model_LR=LogisticRegression()
model_obj=model_LR
model_name='Logistic Regression'
stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # # # 3.Decesion Tree Classifier
# model_DTC=DecisionTreeClassifier()
# model_obj=model_DTC
# model_name='Decesion Tree Classifier'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 4.Random Forest Classifier
# model_RFC=RandomForestClassifier()
# model_obj=model_RFC
# model_name='Random Forest Classifier'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 5.XGBoost Classifier
# model_XGBC=XGBClassifier()
# model_obj=model_XGBC
# model_name='XGBoost Classifier'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 6.Gradient Boosting Classifier
# model_GBC=GradientBoostingClassifier()
# model_obj=model_GBC
# model_name='Gradient Boosting Classifier'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# # 7.XGBoost Random Forest Classifier
# model_XGBRFC=XGBRFClassifier()
# model_obj=model_XGBRFC
# model_name='XGBoost Random Forest Classifier'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

# 8.Support Vector Machine Classifier
# model_SVC=SVC()
# model_obj=model_SVC
# model_name='Support Vector Machine Classifier'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)


# 9.SGD Classifier
# model_sgd = OneVsRestClassifier(SGDClassifier())
# model_obj=model_sgd
# model_name='Stochastic Gradient Descent Classifier'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

#10.Gausian Process Classifier
# model_GPC = GaussianProcessClassifier()
# model_obj=model_GPC
# model_name='Gausian Process Classifier'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

#11.Gausian Process Classifier
# model_KNNC=KNeighborsClassifier()
# model_obj=model_KNNC
# model_name='K Nearst Neighbour Classifier'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

#12 Linear Discriminant Analysis
# model_LDA=LinearDiscriminantAnalysis()
# model_obj=model_LDA
# model_name='Linear Discriminant Analysis'
# stratified_K_fold_validation(model_obj, model_name, process, n_splits, X, y)

#Exporting the results to csv
#df_model_selection.to_csv("Model_statistics.csv",index = False)
df_model_selection

Wall time: 9.77 s


,Process,Model Name,F1 Scores,Range of F1 Scores,Std Deviation of F1 Scores
0,ALl Features,Logistic Regression,"[0.7, 0.7, 0.7, 0.71, 0.71]",0.7-0.71,0.005477


In [18]:
%%time
# Now lets try to get the best split score using StratifiedKFold Cross Validation

#Initialize the algo
model=LogisticRegression()

#Initialize StratifiedKFold Method
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, 
              random_state=1,
              shuffle=True)

#Initialize For Loop 

i=0
for train,test in kfold.split(features,label):
    i = i+1
    X_train,X_test = features[train],features[test]
    y_train,y_test = label[train],label[test]
    
    model.fit(X_train,y_train)
    test_ds_predicted=model.predict(X_test)
    train_ds_predicted=model.predict(X_train)
    
    test_f1_score=round(f1_score(y_true=y_test, y_pred=test_ds_predicted , average='weighted'),2)
    train_f1_score=round(f1_score(y_true=y_train, y_pred=train_ds_predicted , average='weighted'),2)
    
    #print("Train Score: {}, Test score: {}, for Sample Split: {}".format(model.score(X_train,y_train),model.score(X_test,y_test),i))
    print("Train f1-Score: {}, Test f1-score: {}, for Sample Split: {}".format(train_f1_score,test_f1_score,i))
    


Train f1-Score: 0.73, Test f1-score: 0.71, for Sample Split: 1
Train f1-Score: 0.73, Test f1-score: 0.7, for Sample Split: 2
Train f1-Score: 0.73, Test f1-score: 0.7, for Sample Split: 3
Train f1-Score: 0.73, Test f1-score: 0.7, for Sample Split: 4
Train f1-Score: 0.73, Test f1-score: 0.71, for Sample Split: 5
Wall time: 10.3 s


In [19]:
%%time
#Lets extract the Train and Test sample for split 1
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, #n_splits should be equal to no of cv value in cross_val_score
              random_state=1,
              shuffle=True)
i=0
for train,test in kfold.split(features,label):
    i = i+1
    if i == 1:
        X_train,X_test,y_train,y_test = features[train],features[test],label[train],label[test]

#Final Model
model=LogisticRegression()
model.fit(X_train,y_train)

test_ds_predicted=model.predict(X_test)
train_ds_predicted=model.predict(X_train)

test_f1_score=round(f1_score(y_true=y_test, y_pred=test_ds_predicted , average='weighted'),2)
train_f1_score=round(f1_score(y_true=y_train, y_pred=train_ds_predicted , average='weighted'),2)
print("Train f1-Score: {}, Test f1-score: {}".format(train_f1_score,test_f1_score))


train_score=np.round(model.score(X_train,y_train),2)
test_score=np.round(model.score(X_test,y_test),2)
print('Train Accuracy Score is:{} and  Test Accuracy Score:{}'.format(train_score,test_score))

Train f1-Score: 0.73, Test f1-score: 0.71
Train Accuracy Score is:0.73 and  Test Accuracy Score:0.71
Wall time: 1.95 s


## Hyperparametr Tuning with Grid Search

In [20]:
%%time
from sklearn.model_selection import RandomizedSearchCV
param_grid={
            'penalty':['l1', 'l2'],
            'solver':['liblinear'],
            'max_iter':[100,150] 
           }
model=LogisticRegression()
RS = RandomizedSearchCV(estimator=model,param_distributions=param_grid,cv=5,n_iter=5)

RS.fit(features,label)

Wall time: 4min 31s


RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=5,
                   param_distributions={'max_iter': [100, 150],
                                        'penalty': ['l1', 'l2'],
                                        'solver': ['liblinear']})

In [23]:
RS.best_score_

0.7051449910110419

In [24]:
RS.best_estimator_

LogisticRegression(max_iter=150, penalty='l1', solver='liblinear')

In [28]:
model=LogisticRegression(max_iter=150, penalty='l1', solver='liblinear')
model.fit(X_train,y_train)

LogisticRegression(max_iter=150, penalty='l1', solver='liblinear')

In [29]:
%%time
# Confusion Matrix and Classification Report
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_true=label, y_pred=model.predict(features))
CR=classification_report(y_true=label, y_pred=model.predict(features))
print('Confusion Matrix:\n',cm)
print('\n Classification Report:\n',CR)

Confusion Matrix:
 [[80289 32034]
 [30340 81983]]

 Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.71      0.72    112323
           1       0.72      0.73      0.72    112323

    accuracy                           0.72    224646
   macro avg       0.72      0.72      0.72    224646
weighted avg       0.72      0.72      0.72    224646

Wall time: 433 ms


In [ ]:
%%time
#model.predict()
submission=model.predict(test_ohe)

submission=pd.DataFrame(submission,columns=['target'])
submission.insert(0,'id',test['id'])
submission.to_csv('submission.csv',index=False)